In [ ]:
import os
import numpy as np
from datetime import datetime as dt
import datetime
from datetime import timedelta
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sms
import csv
import re

In [ ]:
date = "28.04"
orig_folder = 'C:/Users/signa/_Relkat/data/data_csv/'+date+"/"
dest_folder = 'C:/Users/signa/_Relkat/data/data_csv/'+date+"/"
energy_folder = 'C:/Users/signa/_Relkat/data/Versuche/Versuche '+date+'/'
print(date)

In [ ]:
MD = pd.read_csv('C:/Users/signa/_Relkat/data/data_csv/'+date+"/MD_"+date+".csv",index_col = 0)
print(MD.shape[0])

1. Herausfiltern der betrachteten Größen Hyd, KSS, KSS_HD in eigene Dataframes
- wichtig: schneller Code durch vektorisierte (spaltenweise) Programmierung (möglichst!) ohne Loops 

In [ ]:
pd.set_option('display.max_rows', 70000)
MD = pd.read_csv('C:/Users/signa/_Relkat/data/data_csv/'+date+"/MD_"+date+".csv",index_col = 0)
MD.drop(['uuid','plc_time','sequence_number','value_string','value_bytes','identifier'],axis=1,inplace=True)
MD.rename(columns={'value_integer':'Hyd','value_double':'KSS'},inplace=True)
MD['KSS_HD'] = MD['KSS']
MD['sys_time'] = MD['sys_time'].apply(lambda x: pd.to_datetime(x))
MD['sys_time'] = MD['sys_time'].apply(lambda x: x.tz_localize(None))
MD.reset_index(drop=True,inplace=True)
KSS_HD_filter = re.compile("(DB_KÜHLMITTEL_HOCHDRUCK)")
MD.loc[~MD['label'].str.contains(KSS_HD_filter),"KSS_HD"] = np.nan
MD.loc[MD['label'].str.contains(KSS_HD_filter),"KSS"] = np.nan

MD

In [ ]:
MD_Hyd = MD[MD['label'].str.contains('Hydraulik')]
MD_Hyd.reset_index(drop=True,inplace=True)
MD_red = MD[~MD['label'].str.contains('Hydraulik')]
print(MD_red.shape)
MD_KSS_HD = MD_red[MD_red['label'].str.contains(KSS_HD_filter)]
MD_KSS_HD.reset_index(drop=True,inplace=True)
MD_red = MD_red[~MD_red['label'].str.contains(KSS_HD_filter)]
print(MD_red.shape)
MD.loc[~MD['label'].str.contains(KSS_HD_filter),"KSS_HD"] = np.nan
MD_red.reset_index(inplace=True,drop=True)
MD_KSS = MD_red[MD_red['label'].str.contains('DB_KÜHLMITTEL.Sollwert ')]
MD_KSS.reset_index(drop=True,inplace=True)
MD_red = MD_red[~MD_red['label'].str.contains('DB_KÜHLMITTEL.Sollwert ')]
MD_red.reset_index(drop=True,inplace=True)
print(MD_red.shape)

2. Aggregrierung der Maschinendaten auf Taktebene
- Bestimmung von Taktstart und Taktende das entsprechende Label ("Taktzeit 1 Start") und den entsprechenden boolschen Wert (True: Start, False: Stop)
- Erzeugung von einer Spalte für jede Größe, so dass ein Sample Taktstart, -ende und -dauer, sowie Sollwerte der drei voreingestellten Stellgrößen enthält;

In [ ]:
MD_red['start_time'] = np.datetime64('NaT')
MD_red['end_time'] = np.datetime64('NaT')
MD_red.loc[(MD_red['value_bool']==False),"end_time"] = MD_red['sys_time']
MD_red.loc[(MD_red['value_bool']==True),"start_time"] = MD_red['sys_time']
MD_red = MD_red[['sys_time','start_time','end_time','value_bool']]
MD_red

In [ ]:
pd.set_option('display.max_rows', 100)
cols = ['start_time','end_time','cycle_time','Hyd','KSS','KSS_HD']
df_data = pd.DataFrame(columns=cols)
bins = MD_red['start_time'].dropna()
df_data['start_time'] = bins
df_data['start_time_shift'] = df_data['start_time'].shift(periods=-1)
df_data.reset_index(drop=True,inplace=True)
df_data

In [ ]:
stop_cycle = np.array(MD_red['end_time'][MD_red['end_time'].notnull()],dtype = 'datetime64[ns]')
for x in range(len(stop_cycle)):
    for i in range(len(df_data)):
        if (stop_cycle[x] > np.array(df_data['start_time'])[i])&(stop_cycle[x] < np.array(df_data['start_time_shift'])[i]):        
            df_data.loc[df_data['start_time']==np.array(df_data['start_time'])[i],'end_time'] = stop_cycle[x]
df_data['end_time'] = pd.to_datetime(df_data['end_time'])
df_data.drop('start_time_shift',axis=1,inplace=True)
df_data['cycle_time'] = df_data['end_time'] - df_data['start_time']

In [ ]:
df_data = df_data[~(df_data['cycle_time']<('0 days 00:04:30.749781805'))&~(df_data['cycle_time']>('0 days 00:06:00.749781805'))]
df_data.reset_index(drop=True,inplace=True)
df_data = df_data[df_data['cycle_time'].notnull()]
df_data

In [ ]:
start_bins = np.array(df_data['start_time'])
end_bins = np.array(df_data['end_time'])

# fill in KSS values
df_data.loc[(start_bins < MD_KSS['sys_time'][0])&(end_bins > MD_KSS['sys_time'][0]),"KSS"] = MD_KSS['KSS'][0]
df_data.loc[(start_bins < MD_KSS['sys_time'][1])&(end_bins > MD_KSS['sys_time'][1]),"KSS"] = MD_KSS['KSS'][1]
df_data.loc[(start_bins < MD_KSS['sys_time'][2])&(end_bins > MD_KSS['sys_time'][2]),"KSS"] = MD_KSS['KSS'][2]
df_data.loc[(start_bins < MD_KSS['sys_time'][3])&(end_bins > MD_KSS['sys_time'][3]),"KSS"] = MD_KSS['KSS'][3]
df_data.loc[(start_bins < MD_KSS['sys_time'][4])&(end_bins > MD_KSS['sys_time'][4]),"KSS"] = MD_KSS['KSS'][4]
df_data.loc[(start_bins < MD_KSS['sys_time'][5])&(end_bins > MD_KSS['sys_time'][5]),"KSS"] = MD_KSS['KSS'][5]
df_data.loc[(start_bins < MD_KSS['sys_time'][6])&(end_bins > MD_KSS['sys_time'][6]),"KSS"] = MD_KSS['KSS'][6]
df_data.loc[(start_bins < MD_KSS['sys_time'][7])&(end_bins > MD_KSS['sys_time'][7]),"KSS"] = MD_KSS['KSS'][7]
df_data.loc[(start_bins < MD_KSS['sys_time'][8])&(end_bins > MD_KSS['sys_time'][8]),"KSS"] = MD_KSS['KSS'][8]
df_data.loc[(start_bins < MD_KSS['sys_time'][9])&(end_bins > MD_KSS['sys_time'][9]),"KSS"] = MD_KSS['KSS'][9]
df_data.loc[(start_bins < MD_KSS['sys_time'][10])&(end_bins > MD_KSS['sys_time'][10]),"KSS"] = MD_KSS['KSS'][10]
df_data.loc[(start_bins < MD_KSS['sys_time'][11])&(end_bins > MD_KSS['sys_time'][11]),"KSS"] = MD_KSS['KSS'][11]
df_data.loc[(start_bins < MD_KSS['sys_time'][12])&(end_bins > MD_KSS['sys_time'][12]),"KSS"] = MD_KSS['KSS'][12]
df_data.loc[(start_bins < MD_KSS['sys_time'][13])&(end_bins > MD_KSS['sys_time'][13]),"KSS"] = MD_KSS['KSS'][13]
df_data.loc[(start_bins < MD_KSS['sys_time'][14])&(end_bins > MD_KSS['sys_time'][14]),"KSS"] = MD_KSS['KSS'][14]
df_data.loc[(start_bins < MD_KSS['sys_time'][15])&(end_bins > MD_KSS['sys_time'][15]),"KSS"] = MD_KSS['KSS'][15]
df_data.loc[(start_bins < MD_KSS['sys_time'][16])&(end_bins > MD_KSS['sys_time'][16]),"KSS"] = MD_KSS['KSS'][16]
df_data.loc[(start_bins < MD_KSS['sys_time'][17])&(end_bins > MD_KSS['sys_time'][17]),"KSS"] = MD_KSS['KSS'][17]
df_data.loc[(start_bins < MD_KSS['sys_time'][18])&(end_bins > MD_KSS['sys_time'][18]),"KSS"] = MD_KSS['KSS'][18]
df_data.loc[(start_bins < MD_KSS['sys_time'][19])&(end_bins > MD_KSS['sys_time'][19]),"KSS"] = MD_KSS['KSS'][19]
df_data.loc[(start_bins < MD_KSS['sys_time'][20])&(end_bins > MD_KSS['sys_time'][20]),"KSS"] = MD_KSS['KSS'][20]
df_data.loc[(start_bins < MD_KSS['sys_time'][21])&(end_bins > MD_KSS['sys_time'][21]),"KSS"] = MD_KSS['KSS'][21]
df_data.loc[(start_bins < MD_KSS['sys_time'][22])&(end_bins > MD_KSS['sys_time'][22]),"KSS"] = MD_KSS['KSS'][22]
df_data.loc[(start_bins < MD_KSS['sys_time'][23])&(end_bins > MD_KSS['sys_time'][23]),"KSS"] = MD_KSS['KSS'][23]
df_data.loc[(start_bins < MD_KSS['sys_time'][24])&(end_bins > MD_KSS['sys_time'][24]),"KSS"] = MD_KSS['KSS'][24]
df_data.loc[(start_bins < MD_KSS['sys_time'][25])&(end_bins > MD_KSS['sys_time'][25]),"KSS"] = MD_KSS['KSS'][25]
df_data.loc[(start_bins < MD_KSS['sys_time'][26])&(end_bins > MD_KSS['sys_time'][26]),"KSS"] = MD_KSS['KSS'][26]
df_data.loc[(start_bins < MD_KSS['sys_time'][27])&(end_bins > MD_KSS['sys_time'][27]),"KSS"] = MD_KSS['KSS'][27]
df_data.loc[(start_bins < MD_KSS['sys_time'][28])&(end_bins > MD_KSS['sys_time'][28]),"KSS"] = MD_KSS['KSS'][28]

# fill in KSS_HD values
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][0])&(end_bins > MD_KSS_HD['sys_time'][0]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][0]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][1])&(end_bins > MD_KSS_HD['sys_time'][1]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][1]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][2])&(end_bins > MD_KSS_HD['sys_time'][2]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][2]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][3])&(end_bins > MD_KSS_HD['sys_time'][3]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][3]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][4])&(end_bins > MD_KSS_HD['sys_time'][4]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][4]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][5])&(end_bins > MD_KSS_HD['sys_time'][5]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][5]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][6])&(end_bins > MD_KSS_HD['sys_time'][6]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][6]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][7])&(end_bins > MD_KSS_HD['sys_time'][7]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][7]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][8])&(end_bins > MD_KSS_HD['sys_time'][8]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][8]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][9])&(end_bins > MD_KSS_HD['sys_time'][9]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][9]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][10])&(end_bins > MD_KSS_HD['sys_time'][10]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][10]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][11])&(end_bins > MD_KSS_HD['sys_time'][11]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][11]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][12])&(end_bins > MD_KSS_HD['sys_time'][12]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][12]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][13])&(end_bins > MD_KSS_HD['sys_time'][13]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][13]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][14])&(end_bins > MD_KSS_HD['sys_time'][14]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][14]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][15])&(end_bins > MD_KSS_HD['sys_time'][15]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][15]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][16])&(end_bins > MD_KSS_HD['sys_time'][16]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][16]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][17])&(end_bins > MD_KSS_HD['sys_time'][17]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][17]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][18])&(end_bins > MD_KSS_HD['sys_time'][18]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][18]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][19])&(end_bins > MD_KSS_HD['sys_time'][19]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][19]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][20])&(end_bins > MD_KSS_HD['sys_time'][20]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][20]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][21])&(end_bins > MD_KSS_HD['sys_time'][21]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][21]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][22])&(end_bins > MD_KSS_HD['sys_time'][22]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][22]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][23])&(end_bins > MD_KSS_HD['sys_time'][23]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][23]
df_data.loc[(start_bins < MD_KSS_HD['sys_time'][24])&(end_bins > MD_KSS_HD['sys_time'][24]),"KSS_HD"] = MD_KSS_HD['KSS_HD'][24]

In [ ]:
Hyd_values = []
for i in range(len(start_bins)):
    Hyd_value = np.array((MD_Hyd[(MD_Hyd['sys_time']>start_bins[i])&(MD_Hyd['sys_time']<end_bins[i])]['Hyd']))[0]
    Hyd_values.append(Hyd_value)

df_data['Hyd'] = np.array(Hyd_values)
df_data['Hyd'] = df_data['Hyd'].shift(periods=-1)
df_data.reset_index(drop=True,inplace=True)
df_data

3. Abgleich der ermittelten Datenpunkte mit den Kontrolldatenpunkten aus der Exceltabelle des Versuchsleiters
- Bei gleichen Sollwerteinstellungen in aufeinanderfolgenden Takten, wird dies nicht in den Maschinendaten angezeigt
- Werte können nach dem "Forward-Fill"-Prizip ergänzt werden
- für den 28.4. ergeben sich 83 Datenpunkte

In [ ]:
# Fill in values from table for 18.02.2021
df_data.loc[0,"KSS"] = 50.0
df_data.loc[1,"KSS"] = 42.0
df_data.loc[2,"KSS"] = 42.0
df_data.loc[0,"KSS_HD"] = 50.0
df_data.loc[1,"KSS_HD"] = 33.0
df_data.loc[2,"KSS_HD"] = 33.0
df_data.loc[10,"KSS_HD"] = 33.0
df_data.loc[13,"KSS_HD"] = 33.0
df_data.loc[16,"KSS_HD"] = 33.0
df_data.loc[27,"KSS_HD"] = 33.0
df_data.loc[39,"KSS_HD"] = 33.0
df_data.loc[45,"KSS_HD"] = 22.0
df_data.loc[48,"KSS_HD"] = 22.0
df_data.loc[51,"KSS_HD"] = 22.0
df_data.loc[52,"KSS_HD"] = 22.0
df_data.loc[55,"KSS_HD"] = 22.0
df_data.loc[67,"KSS_HD"] = 33.0
df_data.loc[76,"KSS_HD"] = 22.0
df_data.loc[24,"KSS"] = 28.0
df_data.loc[45,"KSS"] = 42.0
df_data.loc[48,"KSS"] = 42.0
df_data.loc[52,"KSS"] = 28.0
df_data.loc[55,"KSS"] = 28.0
df_data.loc[61,"KSS"] = 35.0
df_data.loc[67,"KSS"] = 28.0

df_data['KSS'].fillna(method='ffill',inplace=True)
df_data['KSS_HD'].fillna(method='ffill',inplace=True)

In [ ]:
df_data = df_data[~df_data['Hyd'].isnull()]
df_data.reset_index(drop=True,inplace=True)
df_data

4. Konkatenierung mit den entsprechenden Energiedaten aus csv-file (Frequenz: 1Hz)
- Aggregierung der Gesamtenergie mit der Funktion np.trapz für jedes Sample
- (Hier im Beispiel werden die gesamten Zeitreihendaten betrachtet, mit Nullen gefüllt und auf eine Länge gebracht und mit im Dataframe abgespeichert)

In [ ]:
keys = [(np.array(df_data['start_time'])[i], np.array(df_data['end_time'])[i])for i in range(len(df_data))]
energy_dict = {key:[]for key in keys}
with open(os.path.join(energy_folder,'energy_'+date+'.csv'),encoding="utf-8") as csvfile:
    print("opening ",os.path.join(energy_folder,'energy_'+date+'.csv' ))
    reader = csv.reader(csvfile, delimiter=';', quotechar='/')
    next(reader)
    next(reader)   
    for line in reader:
        for key in energy_dict:
                        
            if key[0] <= np.datetime64(datetime.datetime.strptime(line[0], '%d.%m.%Y %H:%M:%S')) <= key[1]:
                    energy_dict[key].append(float(line[2].replace(',', '.')))
cols =  ['start_time']+['end_time']+['energy_total']+list(range(int(df_data['cycle_time'].max().total_seconds())+1))
array_total = []
for key in energy_dict:
    energy_points = energy_dict[key]
    total_energy = np.trapz([float(i) for i in energy_points])
    array_total.append([key[0]]+[key[1]]+[total_energy]+energy_dict[key])
df_en_ts = pd.DataFrame(array_total, columns=cols)
df_en_ts.fillna(0,inplace=True)

In [ ]:
df_total = df_data.merge(df_en_ts, on=['start_time','end_time'], how='inner')

In [ ]:
df_total.to_csv(os.path.join(dest_folder,date+'_total.csv'))
print("saving "+(os.path.join(dest_folder,date+'_total.csv')))

In [1]:
%pwd

'C:\\Users\\Anni\\CS\\_BA\\ReLkat\\Alexandra'